In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

search_input = input()
hrefs = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
url = 'https://www.diningcode.com/list.php?query=' + search_input
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
stores = soup.find_all("a", class_="blink")
for store in stores:
    hrefs.append(store['href'].split('rid=')[-1])
for page in range(2,11):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
   }
    data = {
    'type': '',
    'query': search_input,
    'lat': '',
    'lng': '',
    'dis': '',
    'page': str(page),
    'chunk': '10',
    'rn': ''
    }
    response = requests.post('https://www.diningcode.com/2018/ajax/list.php', headers=headers, data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    stores = soup.find_all("a", class_="blink")
    for store in stores:
        hrefs.append(store['href'].split('rid=')[-1])
print(hrefs)

 수원역


['nK1vIhkDdgp8', 'cW8c9YDYkK9N', 'NqYakN0JYWOB', 'EEgchLv8wmXY', 'tTZ6e6QdnifX', 'odtj0fYHmQuq', '3QlBuSnnhog4', 'JxtQheaw1y3E', 'yiXPKpS7xYuu', 'q3gNXHRayoqG', 'Ieh7s0kC9SUk', 'FFEKN2MuiEM0', 'd5jG04QGJIXF', '8xiYHTWf8bVl', 'B0X0U8xl6Oqa', 'rKk3F0b1OuFQ', 'KbC6jVtA7m9s', 'gBnvHRtsdQjK', 'Jes2YIknncoW', 'nSojt9uOADot', '0roQ0CdIqeKn', 'Sjcup6rNX3MQ', 'lw9ArxMysTYk', 'aEizA3a5SZsa', '2IjDoSkprAVQ', 'P2LcklUoI4iy', 'vZemCxouiNgY', 'qGnDdu2F1JrO', 'O5gCN9O7q4Gu', 'FiznRzIRjlLW', 'SGMKG2EdIWM0', 'dCEVnwNm9YvQ', 'G3sGcAh9oYvK', 'LOAzxHUA20ev', 'iFl3bPnUAI9P', 'E01WdAe0zfOp', 'HvDJB1UWdJSh', '4tnD1r6mVDsu', 'pyPXQmVXm8UF', 'FpEQGriaMIhN', 'sCV1IkCZtSPw', 'pIkvkUzWayEY', 'HboXZPegwNFg', 'kgtuufD8CCLT', 'lZsNI9t18L7G', 'Od36MvnLgeox', '0TgivygeXj8q', 'lUkBQOfjPOCV', 'ij7QIYuQXHzH', 'rI0ksYWnSJqQ', 'TwYzR4mxVdQ9', 'ymDDfki9U3oq', 'mYnrRh6VIACv', 'vypcf7KlTi5w', 'RxMKbJPjOlIh', 'nX89S2nnCbMR', 'jVLkloOoVqPG', 'dlnY7S4WScIU', 't2lpB58Gvl6q', 'EhiCcQbaKnmM', 'Wp3r2I7Fadwz', 'uqkRLV9SvXdO', '83eeLO

In [6]:
keyword_list = ['배달', '아침식사', '점심식사', '저녁식사', '식사모임', '술모임', '혼밥',
                '혼술', '회식', '데이트', '기념일', '가족외식', '간식', '숨은맛집',
                '서민적인', '캐주얼한', '고급스러운', '격식있는', '가성비좋은', '푸짐한',
                '조용한', '시끌벅적한', '예쁜', '깔끔한', '이국적/이색적',
                '경관/야경이좋은', '지역주민이찾는']
conv_list = ['무료주차','발렛주차','주차불가','개별룸','대형룸','24시간영업','야외좌석(테라스)','놀이방','애완동물동반','콜키지무료']
purpose_list = ['아침식사','점심식사','저녁식사','식사모임','가족외식','배달','아이동반','다이어트식당',
                '실버푸드','술모임','차모임','혼카페','혼밥','혼술','접대','회식','데이트','기념일','간식']

keyword_dict = {}

front_url = 'https://www.diningcode.com/profile.php?rid='
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
f = open('res_data.csv', 'w')
f.write('Name,Tag,detail\n')
tot_str=''
for code in hrefs:
    res_url = front_url + code
    response = requests.post(res_url, headers=headers2)
    soup = BeautifulSoup(response.text, 'html.parser')
    res_name = soup.find('div',class_='tit-point').get_text().replace('\n', '')
#     print(res_name)
    tag_list=[]
    for tag in soup.find_all('p',class_= 'icon'):
        tag_list.append(tag.get_text())
    tags_str=''
    for tags in tag_list:
        tags = tags.split('(')
        tag_name = tags[0]
        tag_freq = tags[-1][0:-1]
        if tag_name in purpose_list:
            tag_freq = int(tag_freq) * 0.6
        if tag_name in conv_list:
            tag_freq = 0
        if tag_name not in keyword_dict.keys():
            keyword_dict[tag_name] = 5
        else:
            keyword_dict[tag_name] = keyword_dict[tag_name] + int(tag_freq)
        if tag_name in keyword_list:
            tags_str += (tag_name+' ')*int(tag_freq)
            tot_str += tags_str
    detail = soup.find('div',class_='s-list pic-grade').find('div',class_='btxt').get_text().strip().replace(',',' ')
    f.write(res_name+','+tags_str+','+detail+'\n')
f.write(search_input+','+tot_str+','+'details''\n')
f.close()
df_ori = pd.read_csv('res_data.csv',encoding='cp949')
df_ori = df_ori.fillna(' ')
print(df_ori)

                Name                                                Tag  \
0              아다미식당  점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 혼밥 혼밥 ...   
1              닝교초식당  점심식사 점심식사 점심식사 점심식사 저녁식사 저녁식사 저녁식사 혼밥 혼밥 혼밥 데이...   
2              이가꽈배기                                                      
3                훌리오                          저녁식사 이국적/이색적 이국적/이색적 조용한    
4            브리오슈 도레        간식 점심식사 시끌벅적한 시끌벅적한 시끌벅적한 깔끔한 예쁜 서민적인 캐주얼한    
..               ...                                                ...   
96               르레브                                     서민적인 조용한 캐주얼한    
97         탐앤탐스 수원역점                                    가성비좋은 캐주얼한 깔끔한    
98            BHC 치킨                          저녁식사 술모임 캐주얼한 캐주얼한 시끌벅적한    
99   그래인스쿠키 AK플라자수원점                                          예쁜 고급스러운    
100              수원역  점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 점심식사 점...   

                detail  
0     수원역 | 순대국   따로국밥  
1       수원역 | 돈부리   벤또  
2       수원역 | 꽈배기   전병  

In [ ]:
pip uninstall Pillow

In [11]:
import image
import PIL
import stylecloud

print(keyword_dict)
a=stylecloud.gen_stylecloud(text=keyword_dict, icon_name = "fas fa-fish", background_color="black",font_path='C:/Windows/Fonts/malgun.ttf')

ModuleNotFoundError: No module named 'PIL'

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(df_ori['Tag'])
tags_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
similar_index = np.argsort(-tags_similarity)
count_vec = CountVectorizer(ngram_range=(1, 2))
count_matrix = count_vec.fit_transform(df_ori['Tag'])
tags_similarity = cosine_similarity(count_matrix, count_matrix)
similar_index = np.argsort(-tags_similarity)
res_index = df_ori[df_ori['Name']==search_input].index.values
similar_res = similar_index[res_index, :20]
similar_res_index = similar_res.reshape(-1)

print(df_ori.iloc[similar_res_index][['Name','detail']])

                Name                detail
100              구월동               details
12               유타로     구월동 | 오코노미야키   라멘
2           마초쉐프 구월점     구월동 | 스텔라피자   파스타
1             파니노구스토      구월동 | 화덕피자   파스타
0              비스트로J     구월동 | 시금치피자   파스타
21               차이홍    구월동 | 차돌박이짬뽕   중국집
23          탕화쿵푸 마라탕           인천구월동 | 마라탕
24              그라나다         구월동 | 감바스   펍
30   홍콩반점 0410 인천구월점      구월동 | 찹쌀탕수육   짬뽕
5            곰설채 설렁탕     구월동 | 설렁탕   돌솥설렁탕
27              구월오리      구월동 | 오리고기   주물럭
6               겐키라멘       구월동 | 라멘   차슈덮밥
10           보통날의파스타       구월동 | 파스타   리조또
16       등촌샤브칼국수 구월점  인천구월동 | 샤브샤브   버섯매운탕
63        마호가니 인천구월점       구월동 | 콜드브루   카페
11          홍춘천치즈닭갈비        구월동 | 알쌈   닭갈비
67        니뽕내뽕 인천구월점       구월동 | 퓨전짬뽕   짬뽕
7               사곶냉면        구월동 | 까나리   냉면
14              봉추찜닭        구월동 | 찜닭   누룽지
13               은행골       구월동 | 초밥   참치전문
